## Working with structured data
- Extends RDD to a "DataFrame" object
- DataFrames:
    - Contain Row objects
    - Can run SQL queries
    - Has a schema (leading to more effecient storage)
    - Read and write to JSON, Hive, parquet
    - Communicates with JDBC/ODBC, Tableau
    
## Using Spark SQL in python
- `from pyspark.sql import SQLContext, Row`
- `hiveContext = HiveContext(sc)`
- `inputData = spark.read.json(dataFile)`
- `inputData.createOrReplaceTempView("mySctructuredStuff")`
- `myResultDataFrame = hiveContext.sql("SELECT foo FROM bar ORDER BY foobar")`

## More stuff you can do with DataFrames
- `myResultDataFrame.show()`
- `myResultDataFrame.select("someFieldName")`
- `myResultDataFrame.filter(myResultDataFrame("someFieldName" > 200))`
- `myResultDataFrame.groupBy(myResultDataFrame("someFieldName")).mean()`
- `myResultDataFrame.rdd().map(mapperFunction)`

## DataSets
- In Spark 2.0, a DataFrame is really a DataSet of row objects
- DataSets can wrap known, typed data too. Butt this is mostly transparent to you in oython, since python is untyped.
- So - don't sweat this too much with Python. but the Spark 2.0 way is to use DataSets instead of DataFrames when you can.

## Shell Access
- Spark SQL exposes a JDBC/ODBC server (if you built Spark with Hive support)
- Start it with sbin/start-thriftserver.sh
- Listens on port 10000 by default
- Connect using bin/beeline -u jdbc:hive2://localhost:10000
- Viola, you have a SQL shell to Spark SQL
- You can create new tables, or query existing ones that were coached using `hiveCtx.cacheTable("tableName")`

## UDF'S - User Defined Functions
`
from pyspark.sql.types import IntegerType
hiveCtx.registerFunction("square", lambda x: x*x, IntegerType())
df = hiveCtx.sql("SELECT square('someNumericalField') FROM tableName)
`

# Spark-SQL

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

import collections

# Create a SparkSession (Note, the config section is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "../data/temp").appName("SparkSQL").getOrCreate()


In [2]:
def mapper(line):
    fields = line.split(',')
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), age=int(fields[2]), numFriends=int(fields[3]))

lines = spark.sparkContext.textFile("../data/fakefriends.csv")
people = lines.map(mapper)

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are RDDs and support all the normal RDD operations.
for teen in teenagers.collect():
  print(teen)

# We can also use functions instead of SQL queries:
schemaPeople.groupBy("age").count().orderBy("age").show()

spark.stop()


Row(ID=21, age=19, name="b'Miles'", numFriends=268)
Row(ID=52, age=19, name="b'Beverly'", numFriends=269)
Row(ID=54, age=19, name="b'Brunt'", numFriends=5)
Row(ID=106, age=18, name="b'Beverly'", numFriends=499)
Row(ID=115, age=18, name="b'Dukat'", numFriends=397)
Row(ID=133, age=19, name="b'Quark'", numFriends=265)
Row(ID=136, age=19, name="b'Will'", numFriends=335)
Row(ID=225, age=19, name="b'Elim'", numFriends=106)
Row(ID=304, age=19, name="b'Will'", numFriends=404)
Row(ID=341, age=18, name="b'Data'", numFriends=326)
Row(ID=366, age=19, name="b'Keiko'", numFriends=119)
Row(ID=373, age=19, name="b'Quark'", numFriends=272)
Row(ID=377, age=18, name="b'Beverly'", numFriends=418)
Row(ID=404, age=18, name="b'Kasidy'", numFriends=24)
Row(ID=409, age=19, name="b'Nog'", numFriends=267)
Row(ID=439, age=18, name="b'Data'", numFriends=417)
Row(ID=444, age=18, name="b'Keiko'", numFriends=472)
Row(ID=492, age=19, name="b'Dukat'", numFriends=36)
Row(ID=494, age=18, name="b'Kasidy'", numFriends=194)

# Popular Movies DataFrame

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

def loadMovieNames():
    movieNames = {}
    with open("../data/u.item", encoding="ISO-8859-1") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

# Create a SparkSession (the config bit is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "../data/temp").appName("PopularMovies").getOrCreate()

In [7]:
# Load up our movie ID -> name dictionary
nameDict = loadMovieNames()

# Get the raw data
lines = spark.sparkContext.textFile("../data/u.data")
# Convert it to a RDD of Row objects
movies = lines.map(lambda x: Row(movieID =int(x.split()[1])))
# Convert that to a DataFrame
movieDataset = spark.createDataFrame(movies)

# Some SQL-style magic to sort all movies by popularity in one line!
topMovieIDs = movieDataset.groupBy("movieID").count().orderBy("count", ascending=False).cache()

# Show the results at this point:

#  |movieID|count|
#  +-------+-----+
#  |     50|  584|
#  |    258|  509|
#  |    100|  508|
#  ...

topMovieIDs.show()

# Grab the top 10
top10 = topMovieIDs.take(10)

# Print the results
print("\n")
for result in top10:
    # Each row has movieID, count as above.
    print("%s: %d" % (nameDict[result[0]], result[1]))

# Stop the session
spark.stop()


+-------+-----+
|movieID|count|
+-------+-----+
|     50|  583|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
|    286|  481|
|    288|  478|
|      1|  452|
|    300|  431|
|    121|  429|
|    174|  420|
|    127|  413|
|     56|  394|
|      7|  392|
|     98|  390|
|    237|  384|
|    117|  378|
|    172|  367|
|    222|  365|
|    313|  350|
+-------+-----+
only showing top 20 rows



Star Wars (1977): 583
Contact (1997): 509
Fargo (1996): 508
Return of the Jedi (1983): 507
Liar Liar (1997): 485
English Patient, The (1996): 481
Scream (1996): 478
Toy Story (1995): 452
Air Force One (1997): 431
Independence Day (ID4) (1996): 429
